In [ ]:
#!/usr/bin/env python3
import os
import gc
import json
import time
import subprocess
import numpy as np
from openqasm_bridge.v2 import translate

# ====== Lista de parámetros ======
puntos_para_qd = [
    (1.72, 2.24, 1.89),
    (0.3,  1.0,  2.5),
    (2.5,  0.5,  0.9)
]

compiler_path = "/opt/intel/quantum-sdk/docker-intel_quantum_sdk_1.1.1.2024-11-15T22_03_32+00_00/intel-quantum-compiler"
json_path     = "/opt/intel/quantum-sdk/docker-intel_quantum_sdk_1.1.1.2024-11-15T22_03_32+00_00/intel-quantum-sdk-QDSIM.json"

# ✅ Ruta *absoluta* al script run_qd_kernel_once.py
run_script = os.path.join(os.getcwd(), "run_qd_once.py")

ESTADOS = []
TEMP_EXT = [".qasm", ".cpp", ".so", ".ll", ".qs"]

i = 0
for theta1, theta2, theta3 in puntos_para_qd:
    name = f"kernel_{i}"
    print(f"\n🔹 Simulación {i}: ({theta1:.3f}, {theta2:.3f}, {theta3:.3f})")

    # 1) Generar QASM
    qasm = f"""
OPENQASM 2.0;
include "qelib1.inc";
qreg q[1];
rz({theta1}) q[0];
ry({theta2}) q[0];
rz({theta3}) q[0];
"""
    with open(name+".qasm","w") as f:
        f.write(qasm)

    # 2) QASM → C++
    translated = translate(qasm, kernel_name="my_kernel")
    with open(name+".cpp","w") as f:
        for line in translated:
            f.write(line+"\n")

    # 3) Compilar en QD_SIM
    compile_cmd = [
        compiler_path, "-c", json_path,
        "-p", "trivial", "-S", "greedy",
        "-s", f"{name}.cpp", "-o", f"{name}.so"
    ]
    subprocess.run(compile_cmd, check=True)

    # 4) Ejecutar kernel con run_qd_kernel_once.py
    #    ✅ OJO: YA NO CAPTURAMOS stdout → deja fluir logs de "time evolution"
    result = subprocess.run(
        ["python3", run_script, name],
        text=True,
        capture_output=False
    )

    # 5) Ahora sí, volvemos a capturar SOLO la última línea JSON:
    result_json = subprocess.run(
        ["python3", run_script, name],
        capture_output=True,
        text=True
    )

    json_line = result_json.stdout.strip().splitlines()[-1]
    data = json.loads(json_line)
    amps = np.array([complex(r,i) for r,i in zip(data["real"], data["imag"])])

    ESTADOS.append(amps)
    print(f"✅ Estado guardado: {amps}")

    # 6) Limpieza
    for ext in TEMP_EXT:
        try:
            os.remove(name+ext)
        except:
            pass

    gc.collect()
    time.sleep(0.3)
    i += 1

print("\n✅ Fin batch QD_SIM")
print("Total estados:", len(ESTADOS))



🔹 Simulación 0: (1.720, 2.240, 1.890)
                       Intel(R) Quantum SDK (V1.1)
Processing source file:
/workspace/fidelidad_medidas/kernel_0.cpp 
-------------------------------------------------------------------------------
Intermediate representation (IR) output file: 
/workspace/fidelidad_medidas/kernel_0.ll
Generating quantum object file...
Compiling and linking to shared object...
Final shared object: 
/workspace/fidelidad_medidas/kernel_0.so
-------------------------------------------------------------------------------

🚀 Ejecutando kernel: kernel_0.so
✅ Simulador QD listo: QRT_ERROR_SUCCESS
Mon Oct 27 03:29:37 2025
1761535777 time dependent evolution start time
sweep progress: calculation point=0 0%
sweep progress: calculation point=12598 10%
sweep progress: calculation point=25196 20%
sweep progress: calculation point=37794 30%
sweep progress: calculation point=50392 40%
sweep progress: calculation point=62989 50%
sweep progress: calculation point=75587 60%
sweep p